In [1]:
import pandas as pd
import numpy as np
import re
import mysql.connector
import datetime

data_team = mysql.connector.connect(
  host="data-team.c5rldyuxcxmg.ap-south-1.rds.amazonaws.com",
  user="root",
  passwd="ainaa007",
  use_pure=True)

loan_type = [
    'CONSUMER LOAN',
    'PERSONAL LOAN',
    'CREDIT CARD',
    'GOLD LOAN',
    'BUSINESS',
    'TWO-WHEELER LOAN',
    'OTHER',
    'AUTO LOAN',
    'HOUSING LOAN',
]


In [3]:
def get_cibil_account_feature_data(pan="AGZPM8025C"):
    cibil_data = pd.read_sql(f'''SELECT * from lsd_data.cibil_account_etl_data where CUSTOMERID="{pan}"''', data_team)
    cibil_data['dpd'] = np.where(cibil_data['DAYS_PAST_DUE'].isin(['STD', 'XXX', '', 'TYPE:']),0,
                                 np.where(cibil_data['DAYS_PAST_DUE']
                                          .isin(['SUB', 'SMA', 'DBT', 'LSS']),180,
                                          cibil_data['DAYS_PAST_DUE']))
    cibil_data['dpd'] = cibil_data['dpd']*1

    cibil_data['acct_type_cured'] = cibil_data['ACCT_TYPE_CD'].apply(lambda x: re.sub(' +', ' ', x))
    business_loans = [acct for acct in cibil_data.acct_type_cured.unique().tolist() if 'BUSINESS' in acct]
    cibil_data['acct_type_cured'] = np.where(cibil_data['acct_type_cured']\
                                             .isin(business_loans),'BUSINESS', cibil_data['acct_type_cured'])

    features = {}
    cibil_data['REPORT_DT'] = pd.to_datetime(cibil_data['REPORT_DT'], format='%d/%m/%Y', errors='coerce')
    cibil_data['OPEN_DT'] = pd.to_datetime(cibil_data['OPEN_DT'], format='%d/%m/%Y', errors='coerce')
    cibil_data['vintage'] = max((cibil_data['REPORT_DT'] - cibil_data['OPEN_DT']) / np.timedelta64(1, 'M'))


    features['acct_count'] = len(cibil_data)
    features['last_6m_acct_count'] = len(cibil_data[(cibil_data['REPORT_DT'] - cibil_data['OPEN_DT']) \
                                                               / np.timedelta64(1, 'M') <= 6])
    features['last_3m_acct_count'] = len(cibil_data[(cibil_data['REPORT_DT'] - cibil_data['OPEN_DT']) \
                                                               / np.timedelta64(1, 'M') <= 3])
    features['last_1m_acct_count'] = len(cibil_data[(cibil_data['REPORT_DT'] - cibil_data['OPEN_DT']) \
                                                               / np.timedelta64(1, 'M') <= 1])
    features['active_acct_count'] = len(cibil_data[cibil_data['CLOSED_DT'].isna() == True])

    for acct_type in loan_type:
        df_acct = cibil_data[(cibil_data['acct_type_cured'] == acct_type) & (cibil_data['CLOSED_DT'].isna() == True)]
        df_acct['REPORT_DT'] = pd.to_datetime(df_acct['REPORT_DT'], format='%d/%m/%Y')
        df_acct['OPEN_DT'] = pd.to_datetime(df_acct['OPEN_DT'], format='%d/%m/%Y')
        features[acct_type] = {}
        features[acct_type]['count'] = len(df_acct)
        features[acct_type]['disbursed_amount'] = sum(df_acct['ORIG_LOAN_AM'])
        features[acct_type]['curr_balance'] = sum(df_acct['BALANCE_AM'])
        if sum(df_acct['ORIG_LOAN_AM']) > 0:
            features[acct_type]['bal_to_disb'] = sum(df_acct['BALANCE_AM']) / sum(df_acct['ORIG_LOAN_AM'])
        else:
            features[acct_type]['bal_to_disb'] = 0

        if acct_type == 'CREDIT CARD':
            features[acct_type]['total_limit'] = sum(df_acct['CREDIT_LIMIT_AM'])
            if sum(df_acct['CREDIT_LIMIT_AM']) > 0:
                features[acct_type]['utilization'] = sum(df_acct['BALANCE_AM']) / sum(
                    df_acct['CREDIT_LIMIT_AM'])
            else:
                features[acct_type]['utilization'] = 0
        if len(df_acct['dpd']) > 0:
            features[acct_type]['dpd'] = max(df_acct['dpd'])
        else:
            features[acct_type]['dpd'] = 0
    return features

In [4]:
def get_cibil_inq_feature_data(pan="AGZPM8025C"):
    cibil_inq = pd.read_sql(f'''SELECT * from lsd_data.cibil_enquiry_etl_data where CUSTOMERID="{pan}"''', data_team)
    cibil_inq['acct_type_cured'] = cibil_inq['INQ_PURP_CD_4IN'].apply(lambda x: re.sub(' +', ' ', x))
    business_inq = [acct for acct in cibil_inq.acct_type_cured.unique().tolist() if 'BUSINESS' in acct]

    cibil_inq['acct_type_cured'] = np.where(cibil_inq['acct_type_cured']
                                            .isin(business_inq),'BUSINESS', cibil_inq['acct_type_cured'])


    features = {}
    features['total_inq_count'] = len(cibil_inq)
    features['score'] = max(cibil_inq['SCORE'])
    try:
        cibil_inq['REPORT_DT'] = pd.to_datetime(cibil_inq['REPORT_DT'], format = '%d/%m/%Y')
        cibil_inq['INQ_DATE'] = pd.to_datetime(cibil_inq['INQ_DATE'], format = '%d/%m/%Y')
        features['last_6m_inq_count'] = len(cibil_inq[(cibil_inq['REPORT_DT'] \
                                                       - cibil_inq['INQ_DATE'])/np.timedelta64(1,'M') <= 6])
        features['last_3m_inq_count'] = len(cibil_inq[(cibil_inq['REPORT_DT'] \
                                                       - cibil_inq['INQ_DATE'])/np.timedelta64(1,'M') <= 3])
        features['last_1m_inq_count'] = len(cibil_inq[(cibil_inq['REPORT_DT'] \
                                                       - cibil_inq['INQ_DATE'])/np.timedelta64(1,'M') <= 1])
    except:
        features['last_6m_inq_count'] = 0
        features['last_3m_inq_count'] = 0
        features['last_1m_inq_count'] = 0
    for acct_type in loan_type:
        df_acct = cibil_inq[(cibil_inq['acct_type_cured'] == acct_type)]
        features[acct_type] = {}
        features[acct_type]['inq_count'] = len(df_acct)
    return features

In [5]:
print(get_cibil_inq_feature_data())

{'total_inq_count': 29, 'score': 584, 'last_6m_inq_count': 0, 'last_3m_inq_count': 0, 'last_1m_inq_count': 0, 'CONSUMER LOAN': {'inq_count': 0}, 'PERSONAL LOAN': {'inq_count': 9}, 'CREDIT CARD': {'inq_count': 14}, 'GOLD LOAN': {'inq_count': 0}, 'BUSINESS': {'inq_count': 1}, 'TWO-WHEELER LOAN': {'inq_count': 0}, 'OTHER': {'inq_count': 2}, 'AUTO LOAN': {'inq_count': 3}, 'HOUSING LOAN': {'inq_count': 0}}


In [6]:
print(get_cibil_account_feature_data())

{'acct_count': 26, 'last_6m_acct_count': 0, 'last_3m_acct_count': 0, 'last_1m_acct_count': 0, 'active_acct_count': 7, 'CONSUMER LOAN': {'count': 0, 'disbursed_amount': 0, 'curr_balance': 0, 'bal_to_disb': 0, 'dpd': 0}, 'PERSONAL LOAN': {'count': 5, 'disbursed_amount': 898000, 'curr_balance': 93663, 'bal_to_disb': 0.10430178173719376, 'dpd': 900}, 'CREDIT CARD': {'count': 0, 'disbursed_amount': 0, 'curr_balance': 0, 'bal_to_disb': 0, 'total_limit': 0, 'utilization': 0, 'dpd': 0}, 'GOLD LOAN': {'count': 0, 'disbursed_amount': 0, 'curr_balance': 0, 'bal_to_disb': 0, 'dpd': 0}, 'BUSINESS': {'count': 1, 'disbursed_amount': 155000, 'curr_balance': 5519, 'bal_to_disb': 0.035606451612903225, 'dpd': 900}, 'TWO-WHEELER LOAN': {'count': 0, 'disbursed_amount': 0, 'curr_balance': 0, 'bal_to_disb': 0, 'dpd': 0}, 'OTHER': {'count': 1, 'disbursed_amount': 87919, 'curr_balance': 87919, 'bal_to_disb': 1.0, 'dpd': 150}, 'AUTO LOAN': {'count': 0, 'disbursed_amount': 0, 'curr_balance': 0, 'bal_to_disb': 0,

/Users/abhishek/projects/ext/.env/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/abhishek/projects/ext/.env/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
